In [37]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
churn = pd.read_csv('DATA_Customer-Churn.csv')

In [3]:
churn.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [6]:
churn["TotalCharges"] = pd.to_numeric(churn["TotalCharges"], errors='coerce')

In [14]:
churn = churn.dropna()

In [15]:
X = churn[["SeniorCitizen","tenure","TotalCharges","MonthlyCharges"]]
y = churn['Churn']

In [16]:
y = y.apply(lambda x: 1 if x=="Yes" else 0)

In [17]:
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7032, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [29]:
knn_model = KNeighborsClassifier()
param_grid = {'n_neighbors':list(range(1,12))}


In [39]:
grid_search = GridSearchCV(knn_model, param_grid, cv=5,n_jobs=-1,)

In [40]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]})

In [41]:
grid_search.best_params_

{'n_neighbors': 10}

In [57]:
best_params = grid_search.best_params_

In [58]:
model = KNeighborsClassifier(**best_params)

In [59]:
model.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=10)

In [60]:
model.score(X_test,y_test)

0.7912400455062572

In [61]:
import seaborn as sns

In [63]:
tt = sns.load_dataset('titanic')

tt.columns = [i.capitalize() for i in tt.columns]

In [64]:
tt

,Survived,Pclass,Sex,Age,Sibsp,Parch,Fare,Embarked,Class,Who,Adult_male,Deck,Embark_town,Alive,Alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [66]:
X_train, X_test, y_train, y_test = train_test_split(tt.drop(columns=["Alive"]),tt["Alive"])

In [85]:
def clean_missing_values(X_train, X_test, column_names=["Age","Embarked"], drop_columns=["Cabin"]):
    """
    Clean missing values in specified columns in train and test dataframes and drop specified columns.
    Args:
    train_df (pandas.DataFrame): The training dataset.
    test_df (pandas.DataFrame): The test dataset.
    column_names (list): The names of columns to clean missing values.
    drop_columns (list): The names of columns to drop.
    Returns:
    train_df (pandas.DataFrame): The cleaned training dataset.
    test_df (pandas.DataFrame): The cleaned test dataset.
    """
    train_df = X_train
    test_df = X_test
    train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
    train_df['Embarked']=train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])
    #train_df.drop(axis=1, columns=drop_columns,inplace=True)
    test_df['Age'].fillna(train_df['Age'].median(), inplace=True)
    test_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
    #test_df.drop(axis=1, columns=drop_columns,inplace=True)
    X_train = train_df
    X_test = test_df
    return X_train, X_test

In [86]:
X_train_cleaned, X_test_cleaned = clean_missing_values(X_train, X_test)

In [87]:
X_train_cleaned["Embarked"].isna().any()

False

In [84]:
X_train['Embarked'].mode()[0]

'S'